<a href="https://colab.research.google.com/github/yscope75/CS2225.CH2001020/blob/master/Image_captioning_Master_courses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torchvision.datasets as dset
import torchvision.datasets.utils as dset_utils
import torchvision.transforms as transforms
import os

In [2]:
import torch
from torch import nn
import torchvision
from torchsummary import summary

In [ ]:
annotation_folder = os.path.join(os.path.abspath('.') + '/coco/annotations/')
image_folder = os.path.join(os.path.abspath('.') + '/coco/train2014/')
# Download and unzip annotations 
dset_utils.download_and_extract_archive(url='http://images.cocodataset.org/annotations/annotations_trainval2014.zip',
                                        download_root=annotation_folder,
                                        extract_root=annotation_folder,
                                        filename='captions.zip')
# Download and unzion images
dset_utils.download_and_extract_archive(url='http://images.cocodataset.org/zips/train2014.zip',
                                        download_root=image_folder,
                                        extract_root=image_folder,
                                        filename='train2014.zip')

Extracting /content/coco/annotations/captions.zip to /content/coco/annotations/


Extracting /content/coco/train2014/train2014.zip to /content/coco/train2014/


In [3]:
# Setup device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Definition of main model
# Begin with Encoder
class Encoder(nn.Module):
  """
    Encode image input using pre-trained Resnet152 model on imagenet
  """
  def __init__(self, encode_size=64):
    super(Encoder, self).__init__()
    resnet152 = torchvision.models.resnet152(pretrained=True)
    # remove the last two layers and keep the last CNN output 
    modules = list(resnet152.children())[:-2] 
    self.res_encoder = nn.Sequential(*modules) # last output 
    # 

  def forward(self, X_in):
    """
      The forward pass of encoder 
      args:
      - X_in: input data batch of size (batch, 3, Height, weight)
      return: encoded images of size (batch, encode_size, encode_size, 2048)
    """
    e_out = self.res_encoder(X_in) 
  

In [ ]:
resent152 = torchvision.models.resnet152(pretrained=True)

In [ ]:
summary(resent152, input_size=(3,244,244))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 122, 122]           9,408
       BatchNorm2d-2         [-1, 64, 122, 122]             128
              ReLU-3         [-1, 64, 122, 122]               0
         MaxPool2d-4           [-1, 64, 61, 61]               0
            Conv2d-5           [-1, 64, 61, 61]           4,096
       BatchNorm2d-6           [-1, 64, 61, 61]             128
              ReLU-7           [-1, 64, 61, 61]               0
            Conv2d-8           [-1, 64, 61, 61]          36,864
       BatchNorm2d-9           [-1, 64, 61, 61]             128
             ReLU-10           [-1, 64, 61, 61]               0
           Conv2d-11          [-1, 256, 61, 61]          16,384
      BatchNorm2d-12          [-1, 256, 61, 61]             512
           Conv2d-13          [-1, 256, 61, 61]          16,384
      BatchNorm2d-14          [-1, 256,